In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, re
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import folium

import os
for dirname, _, filenames in os.walk('/content/California_Fire_Incidents.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
wf_input = pd.read_csv("/content/California_Fire_Incidents.csv")
wf_input['Started'] = pd.to_datetime(wf_input['Started'].astype(str))

In [ ]:
wf_input.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06T18:30:00Z,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17 15:25:00+00:00,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08T18:30:00Z,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30 15:28:00+00:00,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-07-30T18:00:00Z,NaN,False,True,NaN,NaN,NaN,33.709500,Hwy 243 & Hwy 74 near Mountain Center,-116.728850,False,Mountain Fire,100.0,NaN,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15 13:43:00+00:00,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,31,NaN,NaN,NaN,2013-08-30T08:00:00Z,NaN,False,True,NaN,NaN,NaN,39.120000,"Deadwood Ridge, northeast of Foresthill",-120.650000,False,American Fire,100.0,NaN,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10 16:30:00+00:00,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,56,47.0,8.0,117.0,2013-05-11T06:30:00Z,NaN,False,True,NaN,11.0,10.0,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,0.000000,True,Springs Fire,100.0,2167.0,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02 07:01:00+00:00,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [ ]:
# Some Records may be counted in diffrent counties multiple times
wf = wf_input.drop_duplicates(subset=['Name', 'Started', 'AcresBurned','StructuresDamaged', 'StructuresDestroyed'], keep='first', inplace=False, ignore_index=False).reset_index().drop(columns=['index'])

In [ ]:
wf['StartedMonth'] = [x.month for x in wf['Started']]
monthly_count = wf.groupby(["ArchiveYear","StartedMonth"])['AcresBurned'].count().reset_index()
monthly_count.rename(columns={"AcresBurned": "WildfireCount"}, inplace=True)
monthly_count

,ArchiveYear,StartedMonth,WildfireCount
0,2013,1,1
1,2013,2,2
2,2013,4,4
3,2013,5,28
4,2013,6,24
...,...,...,...
63,2019,7,47
64,2019,8,45
65,2019,9,52
66,2019,10,53


In [ ]:
m = folium.Map(location=[37.160317,-120.621407], tiles="Stamen Terrain", zoom_start=6)
for idx in range(len(wf)):
    folium.Circle(
        location=[wf.loc[idx,'Latitude'], wf.loc[idx,'Longitude']],
        radius=math.sqrt(float(wf.loc[idx,'AcresBurned'])*4047/3.14),
        popup=str(wf.loc[idx,'Name'])+', '+str(wf.loc[idx,'ArchiveYear']),
        color="crimson",
        fill=True,
        fill_color="crimson",
    ).add_to(m)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Spatial distribution of All Recorded Wildfire</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))    
    
m